In [1]:
import time
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
####### Section 1. Set up #######
torch.random.manual_seed(0)
model_id = "./Phi-3-medium-4k-instruct" # please replace with local model path
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
 
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
 
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

Loading checkpoint shards: 100%|██████████| 6/6 [01:35<00:00, 15.97s/it]


In [3]:
####### Section 2. GPU Warm up #######
messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]
output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


 To solve the equation 2x + 3 = 7, you need to isolate the variable x. Here's how you can do it step by step:

1. Subtract 3 from both sides of the equation:
   2x + 3 - 3 = 7 - 3
   2x = 4

2. Divide both sides of the equation by 2:
   2x / 2 = 4 / 2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


In [21]:
####### Section 3. Load data and Inference -> Performance evaluation part #######
start = time.time()
data = load_dataset("cais/mmlu","all")
data = data["test"]
outs = pipe(KeyDataset(data, 'question'), **generation_args)
end = time.time()

In [23]:
#너무 오래 걸려..정확도 측정
####### Section 4. Accuracy (Just for leasderboard) #######
# print("===== Answers =====")
# correct = 0
# for i, out in enumerate(outs):
#     correct_answer = data[i]["answer"]
#     answer = out[0]["generated_text"].lstrip().replace("\n","")
#     if answer == correct_answer:
#         correct += 1
#     print(answer)
 
print("===== Perf result =====")
print("Elapsed_time: ", end-start)
# print(f"Correctness: {correct}/{len(data)}")

===== Perf result =====
Elapsed_time:  3.542080879211426
